<a href="https://colab.research.google.com/github/jiuyuanx/11663-LLM/blob/main/LLM_eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q neo4j

In [ ]:
!pip install -q langchain

In [ ]:
import pandas as pd
import numpy as np
from neo4j import GraphDatabase
from langchain.chains import GraphCypherQAChain
from langchain.graphs import Neo4jGraph

In [ ]:
def run(driver,query): #cypher query runner
    with driver.session() as session:
        return [r for r in session.run(query)]

In [ ]:
url = "neo4j+s://8e7ec38d.databases.neo4j.io"
username="neo4j"
password="xnC4LX_6kXfK08QZVefd7tCqui7lBKxnAr_DeQJKiRY"

graph = Neo4jGraph(
    url=url,
    username=username,
    password=password)

In [ ]:
# run(driver,'.')

In [ ]:
df = pd.read_csv("llm_all_v3.csv")
df.head(5)
df['llama_cleaned_generations'] = df['llama_cleaned_generations'].apply(lambda x: x.replace('[]', '.'))

In [ ]:
df_test = df[df['test']==1]
df_train = df[df['test']==0]
df_test.head(5)

,Unnamed: 0.1,Unnamed: 0,prompt,palm_generations,question,cypher,test,fine_tuned_palm_generations,fine_tuned_llama_generations,fine_tuned_llama_cleaned_generations,llama_generations,llama_cleaned_generations,gpt_generations,gpt_fine_tuned_generations
0,0,0,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper) RETURN count(p),How many papers are in the database?,MATCH (p:paper) RETURN count(p) AS num_papers,1,MATCH (p:paper)\nRETURN COUNT(p),<s> Generate cypher query for the question: Ho...,MATCH (p:paper) RETURN count(p) AS num_papers,"To answer the question ""How many papers are ...","\nMATCH (p:Paper) RETURN p.id, COUNT(p)\n",MATCH (p:paper)\nRETURN count(p),MATCH (p:paper) RETURN count(p) AS num_papers
1,1,1,Given my Neo4j database schema:[<Record nodes=...,MATCH (a:author) RETURN count(a),Count the number of authors.,MATCH (a:Author) RETURN count(a) AS authorCount,1,MATCH (a:author)\nRETURN COUNT(a) AS authorCount,<s> Generate cypher query for the question: Co...,MATCH (a:Author)\nRETURN count(a) AS authorCount,Sure! Here's a Cypher query to count the num...,\nMATCH (a:Author) RETURN COUNT(a)\n,MATCH (a:author)\nRETURN count(a) AS authorCount,MATCH (a:Author) RETURN count(a) AS authorCount
4,4,4,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers cite at least one other paper?,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,1,MATCH (p:paper)-[:paper_cite_paper]->(:paper)...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,"To answer the question ""How many papers cite a...",MATCH (p1:Paper)-[:CITES {num_cites: > 1}]- (p...,MATCH (p1:paper)-[:paper_cite_paper]->(p2:pape...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...
5,5,5,Given my Neo4j database schema:[<Record nodes=...,MATCH (p:paper)-[r:paper_cite_paper]->(p2:pape...,How many papers are cited by at least one othe...,MATCH (other:paper)-[:paper_cite_paper]->(p:pa...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citingP...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,"To answer the question ""How many papers are ci...",MATCH (p:Paper)-[:CITES {num_cites: > 1}]-()\n...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper) ...
6,6,6,Given my Neo4j database schema:[<Record nodes=...,```\nMATCH (p:paper)-[r:paper_cite_paper]->(c:...,How many papers are cited by at least ten othe...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,1,MATCH (p:paper)<-[:paper_cite_paper]-(citing:p...,<s> Generate cypher query for the question: Ho...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...,To find the number of papers that are cited by...,MATCH (p:Paper)-[:CITES {numCitations: > 10}]-...,MATCH (p:paper)-[:paper_cite_paper]->(:paper)\...,MATCH (p:paper)<-[:paper_cite_paper]-(:paper)\...


In [ ]:
df.iloc[0]['gpt_generations']

'MATCH (p:paper)\nRETURN count(p)'

# **test**

In [ ]:
from tqdm.auto import tqdm
driver = GraphDatabase.driver(url, auth=(username, password))
results = np.zeros(6) #6 models runnable cypher
for i in tqdm(range(len(df_test)), "evaluating..."):
  row = df.iloc[i]
  palm2 = row['palm_generations']
  palm2_fine_tuned = row['fine_tuned_palm_generations']
  llama2 = row['llama_cleaned_generations']
  llama2_fine_tuned = row['fine_tuned_llama_cleaned_generations']
  gpt = row['gpt_generations']
  gpt_fine_tuned = row['gpt_fine_tuned_generations']
  cyphers  = [palm2, palm2_fine_tuned, llama2,llama2_fine_tuned, gpt, gpt_fine_tuned]
  for j, cy in enumerate(cyphers):
    try:
      cypher = cy.lower()
      out = run(driver, cypher)
      if out !=[]:
        results[j]+=1
    except:
      pass

results


evaluating...:   0%|          | 0/56 [00:00<?, ?it/s]

array([26., 54.,  9., 50., 46., 54.])

# **test all**

In [ ]:
from tqdm.auto import tqdm
import time
driver = GraphDatabase.driver(url, auth=(username, password))
results_all = np.zeros(6) #6 models runnable cypher
for i in tqdm(range(len(df)), "evaluating..."):
  row = df.iloc[i]
  palm2 = row['palm_generations']
  palm2_fine_tuned = row['fine_tuned_palm_generations']
  llama2 = row['llama_cleaned_generations']
  llama2_fine_tuned = row['fine_tuned_llama_cleaned_generations']
  gpt = row['gpt_generations']
  gpt_fine_tuned = row['gpt_fine_tuned_generations']
  # print(llama2)
  cyphers  = [palm2, palm2_fine_tuned, llama2,llama2_fine_tuned, gpt, gpt_fine_tuned]
  for j, cy in enumerate(cyphers):
    try:
      cypher = cy.lower()
      out = run(driver, cypher)
      if out !=[]:
        results_all[j]+=1
    except:
      pass
  print(i,llama2)
  if i ==130: #reconnect to avoid api error
    time.sleep(100)
    graph = Neo4jGraph(
    url=url,
    username=username,
    password=password)
    driver = GraphDatabase.driver(url, auth=(username, password))


results_all


evaluating...:   0%|          | 0/219 [00:00<?, ?it/s]

0 
MATCH (p:Paper) RETURN p.id, COUNT(p)

1 
MATCH (a:Author) RETURN COUNT(a)

2 
MATCH (c:Conference) RETURN c.name

3 
MATCH (a:Author {name: "Daphne Koller"})-[:AUTHORED_PAPER]->(p:Paper) RETURN p

4 MATCH (p1:Paper)-[:CITES {num_cites: > 1}]- (p2:Paper)
RETURN p1.name, p2.name, COUNT(p1)
5 MATCH (p:Paper)-[:CITES {num_cites: > 1}]-()
RETURN p.name, COUNT(p)
6 MATCH (p:Paper)-[:CITES {numCitations: > 10}]->(otherPaper:Paper)
RETURN otherPaper.name, COUNT(otherPaper)
7 
MATCH (n:DOMAIN) RETURN COUNT(n)

8 
MATCH (a:affiliation) RETURN a

9 
MATCH (n:DOMAIN)
RETURN n.name

10 
MATCH (p:Paper)
WHERE p.domain = 'Computer Science'
RETURN p

11 
MATCH (p:Paper)-[:PRESENTED_AT]->(c:Conference {name: 'AAAI'})
RETURN p.title, c.name

12 
MATCH (n:affiliation) RETURN COUNT(n)

13 .
14 MATCH (p:Paper)-[r:CITES]->(c:Paper)
WHERE r.num_cites > 3
RETURN p.name, r.num_cites
15 
MATCH (c:Conference)
MATCH (p:Paper)
WHERE p.conference = c
RETURN c.name, COUNT(p)

16 
MATCH (d:domain)
WHERE NOT (d) /

ERROR:neo4j:Failed to read from defunct connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


18 MATCH (n:AUTHOR)-[:CITES {num_cites: > 10}]->(p:PAPER)
RETURN n.name, p.name, p.num_cites


ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


...

ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


41 MATCH (a:Author)-[:WRITTEN_PAPER]->(p:Paper)
WHERE COUNT(p) > 5
RETURN a


ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


42 
MATCH (p:Paper)-[:PRESENTED_AT]->(conf:Conference {name: 'AAAI'})
RETURN p.domain



ERROR:neo4j:Unable to retrieve routing information


In [ ]:

for i in tqdm(range(130, len(df)), "evaluating..."):
  row = df.iloc[i]
  palm2 = row['palm_generations']
  palm2_fine_tuned = row['fine_tuned_palm_generations']
  llama2 = row['llama_cleaned_generations']
  llama2_fine_tuned = row['fine_tuned_llama_cleaned_generations']
  gpt = row['gpt_generations']
  gpt_fine_tuned = row['gpt_fine_tuned_generations']
  # print(llama2)
  cyphers  = [palm2, palm2_fine_tuned, llama2,llama2_fine_tuned, gpt, gpt_fine_tuned]

  for j, cy in enumerate(cyphers):
    try:
      cypher = cy.lower()
      out = run(driver, cypher)
      if out !=[]:
        results_all[j]+=1
    except:
      pass
  print(i, llama2)
  # if i in np.arange(30,len(df), 50): #reconnect to avoid api error
  #   time.sleep(100)
  #   graph = Neo4jGraph(
  #   url=url,
  #   username=username,
  #   password=password)
  #   driver = GraphDatabase.driver(url, auth=(username, password))


evaluating...:   0%|          | 0/89 [00:00<?, ?it/s]

130 MATCH (a:Author)-[R]->(b:Author)
WHERE a.domain = 'Machine Learning' AND b.domain = 'Machine Learning'
RETURN a.name, b.name, COUNT(R)
ORDER BY COUNT(R) DESC
LIMIT 1
131 MATCH (a:Author)-[:PUBLISHED]->(p:Paper)
WHERE p.domain = 'Artificial Intelligence'
RETURN a.name
ORDER BY COUNT(p) DESC
LIMIT 1
132 
MATCH (p:Conference)-[:PUBLISHED]->(p:Paper)
RETURN p.citations_count
GROUP BY p.conference_name
SUM p.citations_count

133 MATCH (p:Paper)-[:PUBLISHED]->()
WHERE p.domain = 'Data Mining'
RETURN p.year
ORDER BY p.year DESC
LIMIT 1;
134 
MATCH (a:Author)
WHERE a.fieldOfInterest = 'Computer Vision'
MATCH (a)-[:CO_AUTHOR {role: 'CO-AUTHOR'}]-()
RETURN a.name, COUNT(a)

135 MATCH (p:Paper)-[:CITES {count: > 5}]->(c:Paper)
RETURN p.name, count(c)
136 
MATCH (n:UNIVERSITY)
WHERE n.field = 'Cloud Computing'
RETURN n.name, COUNT(n.id)

137 
MATCH (p:Paper)-[:CITES {cite_text: "learning to rank using gradient descent"}]-()
RETURN p.venue

138 MATCH (n:AUTHOR {INSTITUTION: 'University of Oxfor

ERROR:neo4j:Failed to read from defunct connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


158 
MATCH (p:Paper)-[:PRESENTED_AT]->(a:Affiliation)
WHERE a.name = 'Carnegie Mellon University'
RETURN p.name, p.author, p.venue, p.year



ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


...

179 MATCH (n:CONFERENCE{name: 'SIGGRAPH'})
MATCH (n)-[:PUBLISHED_PAPER {year: 2015 TO 2020}]-(p)
MATCH (p)-[:AUTHOR {name: 'John Doe'}]-(a)
RETURN a.name, p.title, p.author, p.venue, p.year


ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information
ERROR:neo4j:Unable to retrieve routing information


180 
MATCH (p:Paper)-[:PUBLISHED_IN]->(j:Journal)
WHERE j.name = 'ACL' AND p.author = 'Stanford University'
RETURN p.title

181 MATCH (n:UNIVERSITY {name: 'AAAI'})
RETURN n.citations
ORDER BY n.citations DESC
LIMIT 10;
182 MATCH (a:Author)-[:COLLABORATES]->(p:Paper)
WHERE p.conference.name = 'AAAI'
RETURN a.name, COUNT(p) AS num_collaborations
ORDER BY num_collaborations DESC
LIMIT 10;
183 
MATCH (a:Author {name: "Michael I Jordan"})-[:AUTHOR_OF {paper:Paper}]->(p)
MATCH (p)-[:PUBLISHED_IN {domain:Domain}]->(d)
RETURN a.name, p.title, d.name

184 []
185 
MATCH (a:Author {name: 'Daphne Koller'})
MATCH (b:Paper {conference: 'SIGGRAPH'})
MATCH (a)-[R]->(b)
RETURN a.name, b.name, R.type

186 []
187 
MATCH (n:INSTITUTION{name: "University of Toronto"})
MATCH (n)-[:PUBLISHES]->(p:ARTICLE{conference: "ICML"})
RETURN p.main_research_area

188 MATCH (p:Paper)
WHERE p.topic = 'Augmented Reality'
RETURN p.influence
ORDER BY p.influence DESC
LIMIT 5;
189 
MATCH (a:Author)-[:PUBLISHED]->(p:Paper)
W

ERROR:neo4j:Failed to write data to connection ResolvedIPv4Address(('34.121.155.65', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))
ERROR:neo4j:Failed to write data to connection IPv4Address(('8e7ec38d.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.121.155.65', 7687)))


205 
MATCH (p:paper)-[:AUTHOR]->(a:author)
WHERE a.name = 'Stanford University' AND p.conference = 'ICML' AND p.topic = 'Machine Learning' AND p.topic = 'Computational Biology'
RETURN p.id, COUNT(p)

206 
MATCH (p:Paper)-[:PUBLISHED]->()
WHERE p.topic = 'Machine Learning'
RETURN p.published

207 MATCH (a:Author)-[:PUBLISHED]->(p:Publication)
WHERE a.domain = 'Artificial Intelligence'
RETURN a.name, COUNT(p) AS num_publications
ORDER BY num_publications DESC
LIMIT 3;
208 
MATCH (p:Paper)-[:PUBLISHED]->()
WHERE p.title contains "Relational Database"
AND p.publishedDate < Timestamp.now()
RETURN p.title, p.publishedDate

209 MATCH (p:Paper)-[:CITES {numCitations: $numCitations}]-()
RETURN p.name, p.numCitations
ORDER BY p.numCitations DESC
LIMIT 1
210 MATCH (c:Conference)
WHERE (c.name IN ['Feature Vector'])
RETURN c.name, COUNT(p) AS count
ORDER BY count DESC
LIMIT 5;
211 MATCH (n:GRAPHICAL_MODEL {date: > (Date.now() - 10000)))
RETURN n.name, n.description, n.methodology
212 MATCH (a:Auth

## **train**

In [ ]:
# from tqdm.auto import tqdm
# driver = GraphDatabase.driver(url, auth=(username, password))
# results_train = np.zeros(6) #6 models runnable cypher
# for i in tqdm(range(len(df_train)), "evaluating..."):
#   row = df_train.iloc[i]
#   palm2 = row['palm_generations']
#   palm2_fine_tuned = row['fine_tuned_palm_generations']
#   llama2 = row['llama_cleaned_generations']
#   llama2_fine_tuned = row['fine_tuned_llama_cleaned_generations']
#   gpt = row['gpt_generations']
#   gpt_fine_tuned = row['gpt_fine_tuned_generations']
#   cyphers  = [palm2, palm2_fine_tuned, llama2,llama2_fine_tuned, gpt, gpt_fine_tuned]
#   for j, cy in enumerate(cyphers):
#     try:
#       cypher = cy.lower()
#       out = run(driver, cypher)
#       if out !=[]:
#         results_train[j]+=1
#     except:
#       pass
# results_train


evaluating...:   0%|          | 0/163 [00:00<?, ?it/s]

array([ 45.,  87.,   0.,  82.,  93., 147.])

## **Results**

In [ ]:
df_metrics = pd.DataFrame([(results_all-results)/len(df_train), results/len(df_test), results_all/len(df)],
                          columns = ['Palm2', 'Palm2_Fine_tuned', 'Llama2', 'Llama2_LoRA_tuned', 'GPT3.5', 'GPT3.5_Fine_tuned'])

In [ ]:
df_metrics.index = ["Train runnable", "Test runnable",  "All data runnable"]
df_metrics

,Palm2,Palm2_Fine_tuned,Llama2,Llama2_LoRA_tuned,GPT3.5,GPT3.5_Fine_tuned
Train runnable,0.245399,0.822086,0.055215,0.460123,0.552147,0.883436
Test runnable,0.464286,0.964286,0.160714,0.892857,0.821429,0.964286
All data runnable,0.301370,0.858447,0.082192,0.570776,0.621005,0.904110


In [ ]:
!pip install nltk rouge -q
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge


In [ ]:

def calculate_bleu_rouge(reference, candidate):
    """
    Calculate BLEU and ROUGE scores for a pair of strings.

    :param reference: The reference string (ground truth).
    :param candidate: The candidate string (generated text).
    :return: A dictionary containing BLEU and ROUGE scores.
    """
    # Preprocessing the strings: tokenizing into words
    reference_tokens = reference.split()
    candidate_tokens = candidate.split()

    # Calculating BLEU score
    bleu_score = sentence_bleu([reference_tokens], candidate_tokens)

    # Calculating ROUGE score
    rouge = Rouge()
    rouge_score = rouge.get_scores(candidate, reference)[0]

    return np.array([bleu_score,rouge_score['rouge-1']['r'], rouge_score['rouge-2']['r'], rouge_score['rouge-l']['r']])

# Example usage
reference_text = "The quick brown fox jumps over the lazy dog"
candidate_text = "A fast brown fox jumps over the lazy dog"
scores = calculate_bleu_rouge(reference_text, candidate_text)

print(scores)


[0.72597953 0.77777778 0.75       0.77777778]


In [ ]:
from tqdm.auto import tqdm
driver = GraphDatabase.driver(url, auth=(username, password))
scores_all = np.zeros((6,4)) #6 models runnable cypher, 4 metrics
for i in tqdm(range(len(df)), "evaluating..."):
  row = df.iloc[i]
  palm2 = row['palm_generations']
  palm2_fine_tuned = row['fine_tuned_palm_generations']
  llama2 = row['llama_cleaned_generations']
  llama2_fine_tuned = row['fine_tuned_llama_cleaned_generations']
  gpt = row['gpt_generations']
  gpt_fine_tuned = row['gpt_fine_tuned_generations']
  cyphers  = [palm2, palm2_fine_tuned, llama2,llama2_fine_tuned, gpt, gpt_fine_tuned]
  label = row['cypher']
  for j, cy in enumerate(cyphers):
    scores_all[j]+= calculate_bleu_rouge(cy, label)


scores_all = scores_all/len(df)


evaluating...:   0%|          | 0/219 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [ ]:
df_nlp_score = pd.DataFrame(scores_all.T, columns = ['Palm2', 'Palm2_Fine_tuned', 'Llama2', 'Llama2_LoRA_tuned', 'GPT3.5', 'GPT3.5_Fine_tuned'])
df_nlp_score.index = ["BLEU", "ROUGE-1",  "ROUGE-2", "ROUGE-L"]
df_nlp_score

,Palm2,Palm2_Fine_tuned,Llama2,Llama2_LoRA_tuned,GPT3.5,GPT3.5_Fine_tuned
BLEU,0.060464,0.501861,0.017014,0.479566,0.154573,0.786719
ROUGE-1,0.501176,0.794085,0.395649,0.640206,0.595740,0.907770
ROUGE-2,0.218997,0.617748,0.133927,0.549149,0.315950,0.832935
ROUGE-L,0.499025,0.790795,0.392483,0.638954,0.589947,0.905520
